In [152]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist

from random import sample, randint
from collections import defaultdict, Counter
from sklearn.metrics import accuracy_score
from collections import Counter

from __future__ import print_function

In [153]:
train_data_loc = '../data/train-data.txt'
test_data_loc = '../data/test-data.txt'

In [154]:
train_df = pd.read_csv(train_data_loc, header=None, delim_whitespace=True)
test_df = pd.read_csv(test_data_loc, header=None, delim_whitespace=True)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [155]:
train_data = np.array(np.array(train_df)[:,2:], dtype=int)
train_label = np.array(np.array(train_df)[:,1].T, dtype=int)
train_label.resize((train_label.shape[0], 1))
test_data = np.array(np.array(test_df)[:,2:], dtype=int)
test_label = np.array(np.array(test_df)[:,1].T, dtype=int)
test_label.resize((test_label.shape[0], 1))

print(train_data.shape, test_data.shape)
print(train_label.shape, test_label.shape)

(36976, 192) (943, 192)
(36976, 1) (943, 1)


In [ ]:
class MyKNeighborsClassifier:
    
    def __init__(self, k=5):
        self.cache = {}
        self.k = k
        
    def fit(self, train_data, train_label, k=5):
        self.train_data = train_data
        self.train_label = train_label
        self.k = k
    
    def predict(self, test_data):
        d = defaultdict(list)
        res = []
        
        # get k nearest neighbors
        for i, test in enumerate(test_set):
            for j, train in enumerate(self.train_data):
                distance = self.cache.get((i, j)) or dist.euclidean(train, test)
                self.cache[(i, j)] = distance
                d[i].append((j, distance))
            d[i] = sorted(d[i], key=lambda x: x[1])[:self.k]

        # let em vote
        for k, v in d.items():
            labels = Counter([self.train_label[i] for i, _ in v])
            my_label = labels.most_common(n=1)[0][0]
            res.append(my_label)

        return np.array(res)

In [ ]:
res = []
clf = MyKNeighborsClassifier(k=5)
for k in range(2, 100):
    clf.fit(train_data, train_label, k=k)
    pred = clf.predict(test_data)
    score = accuracy_score(test_label, pred)
    res.append(score)
plt.plot(res)
plt.show()

In [192]:
# idea and partial code adapted from Google Developers
# https://www.youtube.com/watch?v=LDRbO9a6XPU

def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    c = Counter()
    for row in rows:
        c[row[-1]] += 1
    return c


def partition(rows, question):
    """Partitions a dataset.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


def gini(rows):
    """Calculate the Gini Impurity for a list of rows.
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = float(counts[lbl]) / len(rows)
        impurity -= prob_of_lbl**2
    return impurity



def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)


def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = float('-inf')
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set([row[col] for row in rows])  # unique values in the column
        for val in values:
            question = Question(col, val)

            true_rows, false_rows = partition(rows, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            current_gain = info_gain(true_rows, false_rows, current_uncertainty)
            if current_gain >= best_gain:
                best_gain, best_question = current_gain, question

    return best_gain, best_question


class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        return val >= self.value



class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)
        
    def get_element(self):
        return list(self.predictions.keys())[0]


class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

        
class MyDecisionTreeClassifier:
    
    def __init__(self):
        self.my_tree = Decision_Node(None, None, None)
    
    def build_tree(rows):
        """Builds the tree.

        Rules of recursion: 
        1) Believe that it works.
        2) Start by checking for the base case (no further information gain).
        3) Prepare for giant stack traces.
        """
        gain, question = find_best_split(rows)
        if gain == 0 or not question:
            return Leaf(rows)

        true_rows, false_rows = partition(rows, question)
        true_branch = build_tree(true_rows)
        false_branch = build_tree(false_rows)
        return Decision_Node(question, true_branch, false_branch)

    
    def fit(self, train_data, train_label):
        training_data = np.concatenate((train_data, train_label), axis=1)
        self.my_tree = build_tree(training_data)
    
    def classify(self, row, node):
        """See the 'rules of recursion' above."""

        # Base case: we've reached a leaf
        if isinstance(node, Leaf):
            return node.get_element()

        # Decide whether to follow the true-branch or the false-branch.
        # Compare the feature / value stored in the node,
        # to the example we're considering.
        if node.question.match(row):
            return self.classify(row, node.true_branch)
        else:
            return self.classify(row, node.false_branch)

    def predict(self, test_data, test_label):
        testing_data = np.concatenate((test_data, test_label), axis=1)
        return np.array([self.classify(row, self.my_tree) for row in testing_data])

    
def dump_tree(tree_id, rowids, colids, clf):
    tree = {
        'rowids': rowids,
        'colids': colids,
        'clf': clf
    }
    f = open('../models/forest_model_test.txt', 'wb')
    f.write(pickle.dumps(tree))
    f.close()
    return tree
    
def do_mdtc(tree_id, selected_rows, selected_cols):
    clf = MyDecisionTreeClassifier()
    clf.fit(train_data[selected_rows,:][:, selected_cols], train_label[selected_rows,:])
    pred = clf.predict(test_data[:,selected_cols], test_label)
    score = accuracy_score(test_label, pred)
    tree = dump_tree(tree_id, selected_rows, selected_cols, clf)
    return pred, tree

In [193]:
all_rowids = list(range(train_data.shape[0]))
all_colids = list(range(train_data.shape[1]))

res = []
trees = []
for iter in range(1000):
    rowids = tuple(sample(all_rowids, randint(200, 1000)))
    colids = tuple(sample(all_colids, randint(8, 20)))
    pred, tree = do_mdtc(iter, rowids, colids)
    res.append(pred)
    trees.append(tree)
serialized_model = pickle.dumps(trees)
f = open('../models/forest model.txt', 'wb')
f.write(serialized_model)
f.close()

In [ ]:
f = open('../models/forest model.txt', 'rb')
models = pickle.loads(f.read())

In [ ]:
res = []
for model in models:
    clf = model['clf']
    rowids, colids = model['rowids'], model['colids']
    pred = clf.predict(test_data[:,colids], test_label)
    res.append(res)
res = np.array(res)
res.shape

In [188]:
clf = model['clf']
rowids, colids = model['rowids'], model['colids']


array([  0,   0, 180, 270,   0,   0, 180,   0,  90, 180, 180,  90, 180,
         0, 180,   0, 180, 180,  90,  90, 180, 180,   0, 270,   0,  90,
        90,   0, 270,  90,  90,  90,  90,   0, 270, 180, 180, 180, 270,
       270,   0,   0, 270, 180,  90, 180, 180,   0, 180,   0,   0,   0,
       180,   0,   0,  90, 270, 180,  90, 270,  90,   0, 180, 270,  90,
        90, 180, 270,  90,   0, 180,   0, 270,  90,   0,   0, 270, 180,
       270, 180,   0, 270,  90, 270,   0, 180,  90, 180, 180,   0,  90,
        90,   0,   0,   0,  90, 270,   0, 270,   0, 270, 180, 180, 270,
         0, 180,  90, 270,   0,  90, 270, 180,  90,  90,  90,  90,   0,
         0,  90,  90, 270, 180,  90,  90, 180, 270,  90,  90,  90,  90,
        90, 180, 270, 180,   0,  90,  90,  90,  90, 180,  90, 180, 180,
        90,  90, 270,   0,   0, 180,   0, 270,  90, 180,  90,   0,  90,
        90,  90,  90,   0,  90, 180,  90, 180, 270,  90,   0, 270, 270,
       270,   0, 180,   0,   0, 180, 270, 180,   0,   0,  90, 27

In [ ]:
my = np.array(res)
final = []
for i in range(my.shape[1]):
    c = Counter(my[:,i])
    val = c.most_common(n=1)[0][0]
    final.append(val)
pred = np.array(final)
accuracy_score(pred, test_label)

In [163]:
all_rowids = list(range(train_data.shape[0]))
all_colids = list(range(train_data.shape[1]))

res = []
for iter in range(1):
    rowids = tuple(sample(all_rowids, randint(200, 1000)))
    colids = tuple(sample(all_colids, randint(8, 20)))
    pred = do_mdtc(rowids, colids)
    res.append(pred)

0.48568398727465534
b'\x80\x03c__main__\nMyDecisionTreeClassifier\nq\x00)\x81q\x01}q\x02X\x07\x00\x00\x00my_treeq\x03c__main__\nDecision_Node\nq\x04)\x81q\x05}q\x06(X\x08\x00\x00\x00questionq\x07c__main__\nQuestion\nq\x08)\x81q\t}q\n(X\x06\x00\x00\x00columnq\x0bK\x01X\x05\x00\x00\x00valueq\x0ccnumpy.core.multiarray\nscalar\nq\rcnumpy\ndtype\nq\x0eX\x02\x00\x00\x00i8q\x0fK\x00K\x01\x87q\x10Rq\x11(K\x03X\x01\x00\x00\x00<q\x12NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x13bC\x08\xa5\x00\x00\x00\x00\x00\x00\x00q\x14\x86q\x15Rq\x16ubX\x0b\x00\x00\x00true_branchq\x17h\x04)\x81q\x18}q\x19(h\x07h\x08)\x81q\x1a}q\x1b(h\x0bK\th\x0ch\rh\x11C\x08\x99\x00\x00\x00\x00\x00\x00\x00q\x1c\x86q\x1dRq\x1eubh\x17h\x04)\x81q\x1f}q (h\x07h\x08)\x81q!}q"(h\x0bK\x07h\x0ch\rh\x11C\x08\x81\x00\x00\x00\x00\x00\x00\x00q#\x86q$Rq%ubh\x17c__main__\nLeaf\nq&)\x81q\'}q(X\x0b\x00\x00\x00predictionsq)ccollections\nCounter\nq*}q+h\rh\x11C\x08\x0e\x01\x00\x00\x00\x00\x00\x00q,\x86q-Rq.K\x1bs\x85q/Rq0sbX\x0c\x00\x00\x00fa

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

def do_sklearn_knn_classification():
    clf = KNeighborsClassifier(n_neighbors=5)
    clf.fit(train_data, train_label.ravel())
    pred = clf.predict(test_data)
    score = accuracy_score(test_label.ravel(), pred)
    print('The accuracy for KNN classification is {}'.format(score))
    
do_sklearn_knn_classification()

def do_sklearn_decision_tree_classification():
    clf = DecisionTreeClassifier()
    clf.fit(train_data, train_label.ravel())
    pred_data = clf.predict(test_data)
    score = accuracy_score(test_label.ravel(), pred_data)
    print('The accuracy for decision tree classification is {}'.format(score))

do_sklearn_decision_tree_classification()

def do_sklearn_adaboost_classification():
    clf = AdaBoostClassifier()
    clf.fit(train_data, train_label.ravel())
    pred_data = clf.predict(test_data)
    score = accuracy_score(test_label.ravel(), pred_data)
    print('The accuracy for adaboost classification is {}'.format(score))
    
do_sklearn_adaboost_classification()